In [54]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

In [58]:
!pip install -U -r /projects/Developer/icesat2_boreal/dps/requirements_main.txt

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# Launch DPS for build_stack.py

In [62]:
# For some reason this is needed to get s3fs to work in ExtractUtils
# this upgrades to 0.3.4 even though we already specify this version in requirements_main...
!pip install s3fs --upgrade

  Using cached s3fs-2022.11.0-py3-none-any.whl (27 kB)
  Attempting uninstall: s3fs
    Found existing installation: s3fs 0.3.4
    Uninstalling s3fs-0.3.4:
      Successfully uninstalled s3fs-0.3.4
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [63]:
import os
import geopandas
import pandas as pd
import glob
import datetime
!pip install xmltodict
import xmltodict
import sys
sys.path.append('/projects/Developer/icesat2_boreal/lib')
import ExtractUtils

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [64]:
import s3fs
s3fs.__version__

'0.3.4'

# Test (locally) the script for DPS

##### To run build_stack.py across a tiled raster dataset you need a bunch of args that we'll gather into a dictionary

s3 you need to have a vector footprint of that dataset

In [4]:
TILE_NUM = 1615

#### Dictionary preparation makes this script very flexible and transferable to another s3 dataset
This dictionary is specific to the ESA Worldcover dataset.  
To run '`build_stack.py` across another dataset, just prepare another dictionary here and everything below should be exactly the same.  

In [5]:
# ESA Worldcover 2020
BUILD_STACK_DICT = {
            #'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
            'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'ID_COL_NAME': 'tile_num',
            'TILE_NUM':TILE_NUM,
            'INDEX_LYR': 'boreal_tiles_v003',
            # Worldcover data is accessed via its footprint, with a 's3_path' col identifying the s3 locations of each tile
            'RASTER_NAME': 'esa_worldcover_v100_2020',
            #'COVAR_TILE_FN': '/projects/my-public-bucket/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'IN_COVAR_S3_COL': 's3_path',
            'OUTDIR': '/projects/my-public-bucket/DPS_ESA_LC',
            'NODATA_VAL': 0,
            'OUTPUT_CLIP_COG_FN':'',
            'CREDENTIALS_FN': None
        }

In [6]:
# INDEX clip shapes should be in equal area
# COVAR_TILE_FN tiles (of raster COGs) should be in 4326
args = f"\
    --in_tile_fn {BUILD_STACK_DICT['INDEX_FN']} \
    --in_tile_id_col {BUILD_STACK_DICT['ID_COL_NAME']} \
    --in_tile_num {BUILD_STACK_DICT['TILE_NUM']} \
    --tile_buffer_m 0 \
    --in_tile_layer {BUILD_STACK_DICT['INDEX_LYR']} \
    -o {BUILD_STACK_DICT['OUTDIR']} \
    --topo_off \
    --covar_src_name {BUILD_STACK_DICT['RASTER_NAME']} \
    --covar_tile_fn {BUILD_STACK_DICT['COVAR_TILE_FN']} \
    --in_covar_s3_col {BUILD_STACK_DICT['IN_COVAR_S3_COL']} \
    --input_nodata_value {BUILD_STACK_DICT['NODATA_VAL']} \
    --clip"

In [8]:
!python /projects/Developer/icesat2_boreal/lib/build_stack.py $args

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

---Running build_stack()---

The covariate's filename(s) intersecting the 0 m bbox for tile id 1615:
 s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N45W084_Map.tif
bbox: [5048522.564673773, 5193303.643996402, 5138522.529666357, 5283303.645755118]
Writing stack as cloud-optimized geotiff: /projects/my-public-bucket/DPS_ESA_LC/esa_worldcover_v100_2020_1615_cog.tif
Clipping to feature polygon...
Orig stack shape:		 (1, 3000, 3000)
Output resolution:		 (30, 30)
Writing img to memory...
/projects/Developer/icesat2_boreal/lib/build_stack.py:94: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of datasets opened in modes other than 'r' will be 

In [8]:
DPS_INPUT_TILE_NUM_LIST = []

# Register DPS algorithm
We need to register a DPS algorithm called 'run_build_stack' before proceeding to the chunks below...

In [47]:
!python /projects/register-algorithm /projects/Developer/icesat2_boreal/dps/alg_3-1-3/algorithm_config.yaml

{
  "algorithm_name": "run_build_stack",
  "code_version": "master",
  "environment_name": "ubuntu",
  "repo_url": "https://github.com/lauraduncanson/icesat2_boreal.git",
  "docker_container_url": "mas.dit.maap-project.org/root/maap-workspaces/base_images/vanilla:dit",
  "queue": "8GB",
  "algorithm_description": "DPS run to create ESA LandCover masks",
  "build_command": "icesat2_boreal/dps/build_command_main.sh",
  "script_command": "icesat2_boreal/dps/alg_3-1-3/run.sh",
  "disk_space": "10GB",
  "algorithm_params": [
    {
      "field": "covar_tile_url",
      "download": true
    },
    {
      "field": "in_tile_url",
      "download": true
    },
    {
      "field": "covar_tile_fn",
      "download": false
    },
    {
      "field": "in_tile_fn",
      "download": false
    },
    {
      "field": "in_tile_id_col",
      "download": false
    },
    {
      "field": "in_tile_num",
      "download": false
    },
    {
      "field": "tile_buffer_m",
      "download": false
    }

# Build a DPS list

In [71]:
HLS_tindex_master_fn = "s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master.csv".replace('s3://maap-ops-workspace/shared/nathanmthomas', '/projects/my-public-bucket')
Topo_tindex_master_fn = "s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv".replace('s3://maap-ops-workspace/shared/nathanmthomas', '/projects/my-public-bucket')

DPS_INPUT_TILE_NUM_LIST = []
for tindex_fn in [HLS_tindex_master_fn, Topo_tindex_master_fn]:
    tindex = pd.read_csv(tindex_fn)
    
    print(len(tindex['tile_num'].values))
    
    if len(tindex['tile_num'].values) > len(DPS_INPUT_TILE_NUM_LIST):
        print(f"{len(tindex['tile_num'].values)} tiles in {tindex_fn}")
        print(f'{os.path.basename(tindex_fn)} has the most tiles. Using this for DPS tiles list.')
        DPS_INPUT_TILE_NUM_LIST = tindex['tile_num'].values


5245
5245 tiles in /projects/my-public-bucket/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master.csv
HLS_tindex_master.csv has the most tiles. Using this for DPS tiles list.
5220


In [72]:
RUN_IRREGULAR_TILES = True

IRREGULAR_TILES_FN = '/projects/my-public-bucket/DPS_tile_lists/HLS_irregular_tiles.csv'

if RUN_IRREGULAR_TILES:
    DPS_INPUT_TILE_NUM_LIST = []
    print("Running DPS on irregular tiles that forces a shape of 3000...")
    df = pd.read_csv(IRREGULAR_TILES_FN)
    tile_num_list = []
    for i in df["IrregularTiles"]:
        tile_num = int(i.split('_')[9])
        DPS_INPUT_TILE_NUM_LIST.append(tile_num)
    print(len(DPS_INPUT_TILE_NUM_LIST))

Running DPS on irregular tiles that forces a shape of 3000...
45


In [74]:
print(DPS_INPUT_TILE_NUM_LIST)

[1941, 2149, 2457, 1448, 1867, 1797, 2010, 1105, 2155, 2779, 1159, 991, 2082, 1731, 2148, 3770, 2301, 36692, 1937, 37142, 37143, 4108, 1334, 36917, 3307, 3951, 3393, 1392, 3486, 2009, 2075, 1798, 2378, 3771, 1273, 1393, 34674, 1050, 1214, 1615, 1938, 2942, 3865, 2223, 2859]


In [14]:
RUN_NAME = 'build_stack_irregulars'

#### Note: make sure the `in_params_dict` coincides with the args of `build_stack.py`

In [48]:
#'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
in_params_dict = {
            'covar_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'in_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'covar_tile_fn': 'footprints_v100_2020_v100_2020_map-s3.gpkg',
            'in_tile_fn': 'boreal_tiles_v003.gpkg',
            'in_tile_id_col': 'tile_num',
            'in_tile_num':"",
            'tile_buffer_m': 0,
            'in_tile_layer': 'boreal_tiles_v003',
            #'output_dir': 'dummy',  # a dummy dir so i dont have to change the .yaml now
            #'topo_off': 'dummy', # functionality to turn off build a 'topo' stack is FALSE by default - so we have to set this flag (which turns OFF topo building) - this is hardcoded in the .sh now - if we want to build a topo stack with this script, need to learn how to set this flag with params dict 
            'covar_src_name': 'esa_worldcover_v100_2020',
            'in_covar_s3_col': 's3_path',
            'input_nodata_value': 0,
            'shape': 3000
            #'clip': 'dummy' # this is hardcoded in the .sh now - if we want to build a topo stack with this script, need to learn how to set this flag with params dict 
        }

In [49]:
in_params_dict

{'covar_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
 'in_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
 'covar_tile_fn': 'footprints_v100_2020_v100_2020_map-s3.gpkg',
 'in_tile_fn': 'boreal_tiles_v003.gpkg',
 'in_tile_id_col': 'tile_num',
 'in_tile_num': '',
 'tile_buffer_m': 0,
 'in_tile_layer': 'boreal_tiles_v003',
 'covar_src_name': 'esa_worldcover_v100_2020',
 'in_covar_s3_col': 's3_path',
 'input_nodata_value': 0,
 'shape': 3000}

## Run a DPS job across the list

In [50]:
%%time
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)
import json

submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):
    
    DPS_num = i+1
    IDENTIFIER = 'run_build_stack'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'nathanmthomas'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    # Update the in_params_dict with th current INPUT_TILE_NUM
    in_params_dict['in_tile_num'] = INPUT_TILE_NUM
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_params_dict
        )
    
    #print(submit_result)
    #break
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 25,50, 100,200,300,400,500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 3750, 4000, 4250, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        
# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
submit_results_df['run_name'] = RUN_NAME
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df


# of input tiles for DPS: 45
DPS run #: 1	| tile num: 1941	| submit status: success	| job id: 4a66e05d-a07f-48cb-85e8-4f76480f079a
DPS run #: 25	| tile num: 3307	| submit status: success	| job id: 9f2c8277-1259-4dda-958d-2e8029b61a41
DPS run #: 45	| tile num: 2859	| submit status: success	| job id: 841538a9-722e-42c2-8663-0dbd632b3c37
Current time:	202212180146
CPU times: user 582 ms, sys: 0 ns, total: 582 ms
Wall time: 9.67 s


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type,run_name
0,success,200,4a66e05d-a07f-48cb-85e8-4f76480f079a,1,1941,2022-12-18 01:46:17.522487,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,1abadad0-1072-4b1f-95ef-75a159c43622,2,2149,2022-12-18 01:46:17.591421,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,8649c52f-e4d4-45b1-949e-1070ac36bb6a,3,2457,2022-12-18 01:46:17.708084,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,af3e95ac-2d78-4760-89c3-1d9b881b0544,4,1448,2022-12-18 01:46:17.825712,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,599f61c5-96c8-4d7e-8d13-0474dd560d59,5,1867,2022-12-18 01:46:18.027760,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,7143ace8-c139-4e4c-b961-400946964cdc,6,1797,2022-12-18 01:46:18.939991,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,8b4b83d3-a16b-4c9c-9248-e59acab009a9,7,2010,2022-12-18 01:46:19.343701,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,c28c372f-792e-49c8-bd67-607dabbcfc5f,8,1105,2022-12-18 01:46:19.494948,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,3032d694-b166-4d74-beac-93a50b658fa1,9,2155,2022-12-18 01:46:19.643927,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars
0,success,200,a13ecdda-ec9f-470c-b576-0989dfd7bb02,10,2779,2022-12-18 01:46:19.803653,1,run_build_stack_ubuntu,nathanmthomas,maap-dps-worker-8gb,build_stack_irregulars


After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [52]:
%%time
LIST_SUBMISSIONS = sorted(glob.glob(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_*.csv'),key=ExtractUtils.func, reverse=True)
for DPS_DATETIME in [nowtime]:
    for fn in LIST_SUBMISSIONS:
        if DPS_DATETIME in fn and not 'job_status' in fn:
            DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
            thentime = fn.split('_')[-1].replace('.csv','')
            print(f'DPS alg:\t\t{DPS_alg_id}')
            print(f'DPS launch time:\t{thentime}')
            z = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
            # Save job status table
            z.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_job_status_{len(z)}_{thentime}.csv')

DPS alg:		run_build_stack
DPS launch time:	202212180146
Count total jobs:	45
Count pending jobs:	0
Count running jobs:	0
Count succeeded jobs:	45
Count failed jobs:	0
% of failed jobs:	Nothing has failed...yet

CPU times: user 507 ms, sys: 0 ns, total: 507 ms
Wall time: 2.03 s


In [179]:
z[z['wps:Status'] =='Succeeded'].tile_num.to_list()

[24389]

In [46]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[43].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '20d88b45-eb25-48e6-b6f0-477ca59f3bb6',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': "+ export HOME=/root\n+ HOME=/root\n+ exec /docker-stats-on-exit-shim _docker_stats.json /app/dps_wrapper.sh /app/icesat2_boreal/dps/alg_3-1-3/run.sh footprints_v100_2020_v100_2020_map-s3.gpkg boreal_tiles_v003.gpkg tile_num 2223 0 boreal_tiles_v003 esa_worldcover_v100_2020 s3_path 0 3000\n+ /app/icesat2_boreal/dps/alg_3-1-3/run.sh footprints_v100_2020_v100_2020_map-s3.gpkg boreal_tiles_v003.gpkg tile_num 2223 0 boreal_tiles_v003 esa_worldcover_v100_2020 s3_path 0 3000\n+ unset PROJ_LIB\n+ mkdir output\n+++ dirname /app/icesat2_boreal/dps/alg_3-1-3/run.sh\n++ cd /app/icesat2_boreal/dps/alg_3-1-3\n++ pwd -P\n+ basedir=/app/icesat2_boreal/

## Update the DPS input tiles list with only the tiles that failed - then run the DPS submit chunk above

In [160]:
DPS_INPUT_TILE_NUM_LIST = z[z['wps:Status'] =='Failed'].tile_num.to_list()
len(DPS_INPUT_TILE_NUM_LIST)
DPS_INPUT_TILE_NUM_LIST

[2917, 980, 25175, 25740, 2981, 3879]

In [162]:
STRAGGLER_TILE_LIST = [3269, 3088, 2917, 2513, 2228, 1554, 1274, 1200, 620, 980, 790, 23828, 24389, 24108, 23501, 23830, 108, 25175, 25458, 25740, 163, 28967, 1172, 1431, 2981, 3531, 4463, 3879, 4207, 4445, 2888, 3790]
DPS_INPUT_TILE_NUM_LIST = STRAGGLER_TILE_LIST
len(DPS_INPUT_TILE_NUM_LIST)

32

In [169]:
FINISHED_LIST = z[z['wps:Status'] =='Succeeded'].tile_num.to_list()
DPS_INPUT_TILE_NUM_LIST = set(STRAGGLER_TILE_LIST) - set(FINISHED_LIST)
len(DPS_INPUT_TILE_NUM_LIST)

6

In [174]:
STRAGGLER_TILE_LIST2 = [3855,24389]
DPS_INPUT_TILE_NUM_LIST = STRAGGLER_TILE_LIST2